In [10]:
import polars as pl

# 使用polars读取当前目录的cb_data.pq数据
df = pl.read_parquet("cb_data.pq")

# 计算前一天跌幅超过5%的可转债
df_with_drop = df.with_columns([
    pl.col('pct_chg').shift(1).over('code').alias('prev_day_pct_chg'),
    pl.col('close').shift(1).over('code').alias('prev_day_close'),
    pl.col('open').alias('next_day_open')
])

# 筛选前一天跌幅超过5%的可转债
drop_bonds = df_with_drop.filter(pl.col('prev_day_pct_chg') < -0.05)

# 计算第二天开盘价高于前一天收盘价的情况
drop_bonds = drop_bonds.with_columns([
    (pl.col('next_day_open') > pl.col('prev_day_close')).alias('is_open_higher'),
    (pl.col('next_day_open') / pl.col('prev_day_close') - 1).alias('overnight_return')
])

# 显示数据形状
print(f"数据形状: {df.shape}")

# 统计结果
total_cases = drop_bonds.shape[0]
if total_cases > 0:
    higher_open_cases = drop_bonds.filter(pl.col('is_open_higher')).shape[0]
    higher_open_probability = higher_open_cases / total_cases
    avg_return = drop_bonds.select(pl.mean('overnight_return')).item()
    
    # 计算按照第二天开盘价卖出的收益
    initial_investment = 100000  # 假设初始投资10万
    final_value = initial_investment * (1 + avg_return)
    profit = final_value - initial_investment
    
    # 假设每次交易都使用全部资金，计算累计收益
    cumulative_return = initial_investment * (1 + avg_return) ** total_cases
    cumulative_profit = cumulative_return - initial_investment
else:
    higher_open_cases = 0
    higher_open_probability = 0
    avg_return = None
    profit = 0
    cumulative_profit = 0

# 显示分析结果
print(f"\n分析结果:")
print(f"前一天跌幅超过5%的可转债总数: {total_cases}个")
print(f"第二天开盘价高于前一天收盘价的概率: {higher_open_probability:.2%}")
print(f"平均隔夜收益率: {avg_return:.4f}" if avg_return is not None else "平均隔夜收益率: N/A")
print(f"是否有收益: {'有' if avg_return is not None and avg_return > 0 else '无'}")

# 显示收益计算结果
if avg_return is not None:
    print(f"\n收益计算:")
    print(f"单次交易平均收益: {profit:.2f}元")
    print(f"假设连续交易{total_cases}次的累计收益: {cumulative_profit:.2f}元")
    print(f"累计收益率: {(cumulative_profit/initial_investment):.2%}")

# 显示部分数据样例
print("\n数据样例:")
display(drop_bonds.select(['code', 'name', 'trade_date', 'prev_day_close', 'next_day_open', 'prev_day_pct_chg', 'is_open_higher', 'overnight_return']).head(10))

if total_cases == 0:
    print("\n没有结果")


数据形状: (574541, 80)

分析结果:
前一天跌幅超过5%的可转债总数: 6254个
第二天开盘价高于前一天收盘价的概率: 53.02%
平均隔夜收益率: 0.0026
是否有收益: 有

收益计算:
单次交易平均收益: 264.58元
假设连续交易6254次的累计收益: 1502028906491.59元
累计收益率: 1502028906.49%

数据样例:


code,name,trade_date,prev_day_close,next_day_open,prev_day_pct_chg,is_open_higher,overnight_return
str,str,datetime[ns],f64,f64,f64,bool,f64
"""110033.SH""","""国贸转债""",2018-02-12 00:00:00,113.12,113.98,-0.061323,true,0.007603
"""110033.SH""","""国贸转债""",2021-09-28 00:00:00,114.71,115.03,-0.111464,true,0.00279
"""110034.SH""","""九州转债""",2018-01-30 00:00:00,109.76,109.62,-0.052241,false,-0.001276
"""110038.SH""","""济川转债""",2018-06-20 00:00:00,115.99,115.99,-0.060658,false,0.0
"""110038.SH""","""济川转债""",2018-08-16 00:00:00,114.42,114.8,-0.066188,true,0.003321
"""110038.SH""","""济川转债""",2021-11-16 00:00:00,121.19,122.51,-0.132125,true,0.010892
"""110038.SH""","""济川转债""",2022-02-15 00:00:00,127.89,129.0,-0.052316,true,0.008679
"""110038.SH""","""济川转债""",2022-09-15 00:00:00,113.911,113.86,-0.057364,false,-0.000448
"""110039.SH""","""宝信转债""",2018-06-20 00:00:00,114.23,114.23,-0.095781,false,0.0


In [1]:
import polars as pl

# 使用polars读取当前目录的cb_data.pq数据
df = pl.read_parquet("cb_data.pq")

In [2]:
df

name,pre_close,open,high,low,close,limit,close_ma_5,bias_5,pct_chg,adj_factor,vol,vol_5,amount,amount_5,volatility,code_stk,name_stk,pre_close_stk,open_stk,high_stk,low_stk,close_stk,pct_chg_stk,adj_factor_stk,vol_stk,amount_stk,pe_ttm,pb,ps_ttm,dv_ratio,total_share,float_share,total_mv,circ_mv,debt_to_assets,volatility_stk,…,remain_size,remain_cap,turnover,turnover_5,cap_mv_rate,list_date,list_days,conv_start_date,left_conv_start_days,conv_end_date,left_years,ytm,pure_value,bond_prem,option_value,theory_value,theory_bias,rating,yy_rating,orgform,area,industry_1,industry_2,industry_3,maturity_put_price,maturity,popularity_ranking,pct_chg_5,pct_chg_5_stk,alpha_pct_chg_5,theory_conv_prem,mod_conv_prem,open_pct_chg,high_pct_chg,low_pct_chg,code,trade_date
str,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,date,i64,date,i64,date,f64,f64,f64,f64,f64,f64,f64,cat,str,str,str,str,str,str,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,str,datetime[ns]
"""格力转债""",105.28,105.3,106.18,105.3,105.4,0,105.084,0.003007,0.00114,1.075947,9075.0,26222.0,958.0068,2759.1957,0.103224,"""600185.SH""","""格力地产""",5.68,5.66,5.89,5.66,5.77,0.015845,0.862552,123240.77,71342.62,18.2321,1.5625,3.61,0.003466,206007.7575,206007.7575,118.866476,118.866476,null,0.294814,…,9.77498,10.302829,0.009298,0.026851,0.086676,2015-01-13,1086,2015-06-30,-917,2019-12-24,1.978082,0.010109,96.481599,0.092436,9.041548,105.523147,-0.001168,"""AA""","""""","""地方国有企业""","""广东""","""房地产""","""房地产开发""","""房地产开发""",106.0,5.0,0,-0.001421,0.0322,-0.033621,0.212592,0.109931,0.00019,0.008549,0.00019,"""110030.SH""",2018-01-02 00:00:00
"""格力转债""",105.4,105.4,105.72,105.4,105.46,0,105.104,0.003387,0.000569,1.07656,9783.0,32295.0,1031.8907,3400.2009,0.103226,"""600185.SH""","""格力地产""",5.77,5.74,5.83,5.7,5.79,0.003466,0.865541,112470.22,64885.787,18.2952,1.5679,3.6225,0.003454,206007.7575,206007.7575,119.278492,119.278492,null,0.294711,…,9.77498,10.308694,0.01001,0.033065,0.086425,2015-01-13,1087,2015-06-30,-918,2019-12-24,1.975342,0.00983,96.551329,0.092269,9.144338,105.695667,-0.002235,"""AA""","""""","""地方国有企业""","""广东""","""房地产""","""房地产开发""","""房地产开发""",106.0,5.0,0,0.000949,0.032086,-0.031136,0.217011,0.101694,0.0,0.003036,0.0,"""110030.SH""",2018-01-03 00:00:00
"""格力转债""",105.46,105.5,105.73,105.3,105.6,0,105.258,0.003249,0.001328,1.077989,8676.0,38915.0,915.2612,4099.5547,0.103057,"""600185.SH""","""格力地产""",5.79,5.78,5.84,5.74,5.8,0.001727,0.867036,86917.89,50287.495,18.3269,1.5706,3.6288,0.003448,206007.9508,206007.9508,119.484611,119.484611,null,0.294694,…,9.77498,10.322379,0.008867,0.039825,0.086391,2015-01-13,1088,2015-06-30,-919,2019-12-24,1.972603,0.009159,96.530032,0.09396,9.190142,105.720175,-0.001138,"""AA""","""""","""地方国有企业""","""广东""","""房地产""","""房地产开发""","""房地产开发""",106.0,5.0,0,0.007345,0.048825,-0.041479,0.219725,0.098455,0.000379,0.00256,-0.001517,"""110030.SH""",2018-01-04 00:00:00
"""格力转债""",105.6,106.08,109.86,105.71,109.34,0,106.216,0.029412,0.035417,1.116168,47628.0,80400.0,5166.3701,8621.316,0.109005,"""600185.SH""","""格力地产""",5.8,5.82,6.26,5.82,6.15,0.060345,0.919357,506597.49,307682.959,19.4328,1.6654,3.8477,0.003252,206007.9508,206007.9508,126.69489,126.69489,null,0.300755,…,9.77498,10.687963,0.048338,0.081856,0.08436,2015-01-13,1089,2015-06-30,-920,2019-12-24,1.969863,-0.008651,96.561903,0.132331,11.815957,108.377859,0.0088,"""AA""","""""","""地方国有企业""","""广东""","""房地产""","""房地产开发""","""房地产开发""",106.0,5.0,0,0.045815,0.110108,-0.064293,0.186283,0.100907,0.004545,0.040341,0.001042,"""110030.SH""",2018-01-05 00:00:00
"""格力转债""",109.34,109.31,111.89,108.73,109.76,0,107.112,0.024722,0.003841,1.120455,31285.0,106447.0,3455.6852,11527.214,0.108986,"""600185.SH""","""格力地产""",6.15,6.13,6.44,6.08,6.27,0.019512,0.937296,565535.31,356757.949,19.812,1.6979,3.9228,0.00319,206007.9508,206007.9508,129.166985,129.166985,null,0.301309,…,9.77477,10.728788,0.0322

In [49]:
# 按照日期分组，计算close和conv_prem的排名
result_df = df.with_columns([
    pl.col('close')
        .rank(descending = True)  
        .over('trade_date')
        .alias('rank_close'),
    
    pl.col('conv_prem')
        .rank(descending = True)  
        .over('trade_date')
        .alias('rank_conv_prem')
])

# 计算score（两个排名相加）
result_df = result_df.with_columns([
    (pl.col('rank_close') + pl.col('rank_conv_prem')).alias('score')
])

# 提取每天score排名前十的数据
top10_df = result_df.sort(['trade_date', 'score'], descending=[False, True]).group_by('trade_date').head(10)
# 显示结果
print("按日期分组后的排名结果及每日score前十:")
display(top10_df.select(['code', 'trade_date', 'close', 'rank_close', 'conv_prem', 'rank_conv_prem', 'score']).head(10))

按日期分组后的排名结果及每日score前十:


code,trade_date,close,rank_close,conv_prem,rank_conv_prem,score
str,datetime[ns],f64,f64,f64,f64,f64
"""128023.SZ""",2018-01-02 00:00:00,95.24,34.0,0.106013,20.0,54.0
"""128019.SZ""",2018-01-02 00:00:00,95.5,32.0,0.108648,19.0,51.0
"""128018.SZ""",2018-01-02 00:00:00,95.4,33.0,0.118483,17.0,50.0
"""128022.SZ""",2018-01-02 00:00:00,104.2,18.0,0.050502,31.0,49.0
"""128025.SZ""",2018-01-02 00:00:00,98.202,26.0,0.093539,22.0,48.0
"""123002.SZ""",2018-01-02 00:00:00,112.122,9.0,-0.011711,37.0,46.0
"""128014.SZ""",2018-01-02 00:00:00,105.766,16.0,0.055082,30.0,46.0
"""113503.SH""",2018-01-02 00:00:00,101.8,20.0,0.07289,25.0,45.0
"""127004.SZ""",2018-01-02 00:00:00,93.5,35.0,0.30541,9.0,44.0


In [54]:
# 选择特定日期的数据
specific_date = "2023-02-24"  # 由于数据中没有2025-01-10，选择了数据中存在的日期

# 筛选该日期的数据
specific_date_df = top10_df.filter(pl.col('trade_date').dt.date() == pl.lit(specific_date).str.to_date())

# 显示该日期的数据
print(f"\n{specific_date}日的可转债数据:")
display(specific_date_df.select(['trade_date',"code", "name", "close", "conv_prem", "rank_close", "rank_conv_prem", "score"]))


2023-02-24日的可转债数据:


trade_date,code,name,close,conv_prem,rank_close,rank_conv_prem,score
datetime[ns],str,str,f64,f64,f64,f64,f64
2023-02-24 00:00:00,"""113044.SH""","""大秦转债""",109.687,0.085529,421.0,427.0,848.0
2023-02-24 00:00:00,"""128114.SZ""","""正邦转债""",96.936,0.174573,474.0,357.0,831.0
2023-02-24 00:00:00,"""128014.SZ""","""永东转债""",110.8,0.116421,411.5,405.0,816.5
2023-02-24 00:00:00,"""113011.SH""","""光大转债""",104.797,0.182054,457.0,349.0,806.0
2023-02-24 00:00:00,"""110043.SH""","""无锡转债""",112.308,0.113194,396.0,409.0,805.0
2023-02-24 00:00:00,"""113595.SH""","""花王转债""",108.362,0.15862,429.0,368.0,797.0
2023-02-24 00:00:00,"""128100.SZ""","""搜特转债""",92.263,0.220007,477.0,301.0,778.0
2023-02-24 00:00:00,"""128034.SZ""","""江银转债""",114.833,0.110768,364.0,412.0,776.0
2023-02-24 00:00:00,"""128048.SZ""","""张行转债""",117.2,0.094672,333.5,424.0,757.5


In [12]:
def get_top_bonds_by_score(df, config):
    """
    根据配置文件计算多因子排名，并获取每个交易日得分最高的可转债
    
    参数:
    df (polars.DataFrame): 输入的数据框
    config (dict): 策略配置参数，包含以下关键字:
        - start_date (str): 起始日期 YYYY-MM-DD
        - end_date (str): 结束日期 YYYY-MM-DD
        - top_n (int): 每天选择的转债数量
        - strategy_params (dict): 策略参数，包含:
            - indicators (list): 用于排名的指标列名列表
            - weights (list): 对应指标的权重列表 (-1表示负相关，1表示正相关)
            - filters (dict): 前置筛选条件，如 {"left_years": [">", 0.5]}
    
    返回:
    tuple: (top_n_df, tracking_df)
        - top_n_df (polars.DataFrame): 包含每个交易日得分最高的N只可转债的DataFrame
        - tracking_df (polars.DataFrame): 包含每个交易日需要关注的所有可转债的DataFrame（当天TOPN和昨天TOPN的并集）
    """
    # 解析配置参数
    start_date = config.get("start_date")
    end_date = config.get("end_date")
    top_n = config.get("top_n", 10)
    strategy_params = config.get("strategy_params", {})
    indicators = strategy_params.get("indicators", [])
    weights = strategy_params.get("weights", [])
    filters = strategy_params.get("filters", {})
     
    # 1. 日期过滤
    filtered_df = df
    filtered_df = filtered_df.filter(
        pl.col('trade_date') >= pl.lit(start_date).str.to_datetime()
    )
    filtered_df = filtered_df.filter(
        pl.col('trade_date') <= pl.lit(end_date).str.to_datetime()
    )
    
    # 2. 应用前置过滤条件
    for column, condition in filters.items():
        operator, value = condition
        if operator == ">":
            filtered_df = filtered_df.filter(pl.col(column) > value)
        elif operator == ">=":
            filtered_df = filtered_df.filter(pl.col(column) >= value)
        elif operator == "<":
            filtered_df = filtered_df.filter(pl.col(column) < value)
        elif operator == "<=":
            filtered_df = filtered_df.filter(pl.col(column) <= value)
        elif operator == "==":
            filtered_df = filtered_df.filter(pl.col(column) == value)
        elif operator == "!=":
            filtered_df = filtered_df.filter(pl.col(column) != value)
    
    # 3. 计算各个指标的排名
    rank_expressions = []
    for i, indicator in enumerate(indicators):        
        # 根据权重确定排序方向
        # 负权重(-1)表示较小值更好，使用descending=True获取更高排名
        # 正权重(1)表示较大值更好，使用descending=False获取更高排名
        weight = weights[i]
        descending = weight < 0
        
        rank_expr = (
            pl.col(indicator)
                .rank(descending=descending)
                .over('trade_date')
                .alias(f'rank_{indicator}')
        )
        rank_expressions.append(rank_expr)
    
    # 4. 添加排名列
    ranked_df = filtered_df.with_columns(rank_expressions)
    
    # 5. 计算综合得分 (权重绝对值 * 排名，然后相加)
    score_expr = None
    for i, indicator in enumerate(indicators):
        weight_abs = abs(weights[i])
        if score_expr is None:
            score_expr = (pl.col(f'rank_{indicator}') * weight_abs)
        else:
            score_expr = score_expr + (pl.col(f'rank_{indicator}') * weight_abs)
    
    ranked_df = ranked_df.with_columns([
        score_expr.alias('score')
    ])
    
    # 6. 根据分组获取每个日期得分最高的前N条记录
    # 由于我们的分数是根据排名计算的，分数越小表示综合排名越靠前
    top_n_df = ranked_df.sort(['trade_date', 'score'], descending=[False, True]).group_by('trade_date').head(top_n)
    
    # 7. 创建每天需要关注的股票DataFrame（当天TOPN和昨天TOPN的并集）
    # 简化版本 - 直接计算每天需要关注的代码
    
    # 获取今日TopN股票代码
    today_codes = top_n_df.select(['trade_date', 'code'])
    
    # 创建日期映射（今天对应昨天）
    dates = filtered_df.select('trade_date').unique().sort('trade_date')
    date_pairs = dates.with_columns([
        pl.col('trade_date'),
        pl.col('trade_date').shift(1).alias('prev_date')
    ]).filter(pl.col('prev_date').is_not_null())
    
    # 获取昨日TopN股票代码
    yesterday_codes = date_pairs.join(
        top_n_df.select(['trade_date', 'code']),
        left_on='prev_date',
        right_on='trade_date'
    ).select(['trade_date', 'code'])
    
    # 合并今日和昨日的代码（取并集）
    all_tracking_codes = pl.concat([today_codes, yesterday_codes]).unique()
    
    # 获取需要关注的所有股票的完整数据
    tracking_df = filtered_df.join(
        all_tracking_codes,
        on=['trade_date', 'code'],
        how='inner'
    )
    
    return top_n_df, tracking_df

In [5]:
config = {
        "data_path": "data/cb_data.pq",
        "start_date": "2018-01-02",
        "end_date": "2024-12-31",
        "initial_capital": 1000000.0,
        "strategy_type": "custom",
        "top_n": 10,
        "name": "自定义策略",
        "output_dir": "results/custom",
        "strategy_params": {
            "indicators": ["close","conv_prem"],
            "weights": [-1.0, -1.0],
            "filters": {
                "left_years": [">", 0.5]
            }
        }
    }

In [13]:
a,b = get_top_bonds_by_score(df = df, config=config)

In [15]:
display(b.sort("trade_date").head(20))

name,pre_close,open,high,low,close,limit,close_ma_5,bias_5,pct_chg,adj_factor,vol,vol_5,amount,amount_5,volatility,code_stk,name_stk,pre_close_stk,open_stk,high_stk,low_stk,close_stk,pct_chg_stk,adj_factor_stk,vol_stk,amount_stk,pe_ttm,pb,ps_ttm,dv_ratio,total_share,float_share,total_mv,circ_mv,debt_to_assets,volatility_stk,…,remain_size,remain_cap,turnover,turnover_5,cap_mv_rate,list_date,list_days,conv_start_date,left_conv_start_days,conv_end_date,left_years,ytm,pure_value,bond_prem,option_value,theory_value,theory_bias,rating,yy_rating,orgform,area,industry_1,industry_2,industry_3,maturity_put_price,maturity,popularity_ranking,pct_chg_5,pct_chg_5_stk,alpha_pct_chg_5,theory_conv_prem,mod_conv_prem,open_pct_chg,high_pct_chg,low_pct_chg,code,trade_date
str,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,date,i64,date,i64,date,f64,f64,f64,f64,f64,f64,f64,cat,str,str,str,str,str,str,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,str,datetime[ns]
"""广汽转债""",116.98,116.81,117.8,116.48,117.2,0,115.758,0.012457,0.001881,1.174018,75668.0,293813.0,8862.814,33908.6687,0.161293,"""601238.SH""","""广汽集团""",24.66,24.68,24.98,24.67,24.9,0.009732,1.257864,125155.37,310337.86,18.8074,2.7101,2.7302,0.010785,728253.5012,431584.454,1813.351218,1074.64529,null,0.226777,…,25.5301,29.921278,0.02962,0.114749,0.016501,2016-02-04,699,2016-07-22,-529,2022-01-21,4.057534,-0.014717,89.787412,0.305305,40.347762,130.135174,-0.110368,"""AAA""","""""","""地方国有企业""","""广东""","""交运设备""","""汽车""","""乘用车""",106.0,6.0,0,0.013665,0.012607,0.001059,0.032341,-0.032611,-0.001453,0.00701,-0.004274,"""113009.SH""",2018-01-02 00:00:00
"""泰晶转债""",100.0,108.0,108.5,101.22,101.8,0,101.8,0.0,0.018,1.018,95058.0,95058.0,9752.4067,9752.4067,0.0,"""603738.SH""","""泰晶科技""",24.28,24.24,24.3,23.85,24.11,-0.007002,0.992998,4464.7,10738.286,42.3245,4.8027,5.915,0.007807,11335.6,4195.6,27.330132,10.115592,null,0.389912,…,2.15,2.1887,0.44558,0.44558,0.080084,2018-01-02,1,2018-06-21,170,2023-12-14,5.953425,0.018496,80.343056,0.267067,41.709215,122.05227,-0.198942,"""AA""","""""","""大型民企""","""湖北""","""电子设备""","""电子元件""","""电子元件""",108.0,6.0,0,0.0,0.0,0.0,0.099231,-0.026341,0.08,0.085,0.0122,"""113503.SH""",2018-01-02 00:00:00
"""国祯转债""",111.11,111.58,112.79,110.1,112.122,0,111.6124,0.004566,0.009108,1.12122,33541.6,312162.0,3754.569476,35032.116332,0.832273,"""300388.SZ""","""国祯环保""",23.79,23.9,24.19,23.63,23.87,0.003363,0.998745,39522.29,94603.297,48.035,4.2981,3.6757,0.004189,30566.8467,30016.8916,72.963063,71.65032,null,0.335835,…,5.97,6.6936834,0.056091,0.524744,0.091741,2017-12-25,9,2018-05-30,148,2023-11-23,5.89589,-0.002355,78.655411,0.425484,52.218088,130.873499,-0.167242,"""AA""","""""","""央企子公司""","""安徽""","""公用事业""","""水务""","""水务""",106.0,6.0,0,-0.014745,0.011869,-0.026615,0.039096,-0.050807,0.00423,0.01512,-0.00909,"""123002.SZ""",2018-01-02 00:00:00
"""模塑转债""",93.258,93.26,93.749,93.26,93.5,0,93.3796,0.001289,0.002595,0.935,16429.3,33045.7,1536.967947,3088.163046,0.243866,"""000700.SZ""","""模塑科技""",5.67,5.68,5.74,5.67,5.73,0.010582,0.769128,43775.34,24978.793,27.0877,1.3215,1.1775,0.019197,71720.7902,69738.3762,41.096013,39.96009,null,0.27797,…,8.136599,7.60772,0.020203,0.040628,0.185121,2017-06-26,191,2017-12-08,-25,2023-06-01,5.416438,0.040729,85.021714,0.099719,16.159787,101.181501,-0.082155,"""AA""","""""","""大型民企""","""江苏""","""交运设备""","""汽车""","""汽车零部件""",110.0,6.0,0,-0.001911,0.038043,-0.039954,0.318709,-0.013299,0.000021,0.005265,0.000021,"""127004.SZ""",2018-01-02 00:00:00
"""永东转债""",104.092,104.197,106.197,104.193,105.766,0,104.379,0.013288,0.016082,1.05766,3371.9,28749.3,355.862794,3007.142976,0.224558,"""002753.SZ""","""永东股份""",20.0,20.0,20.6,19.91,20.51,0.0255,1.263009,29180.36,59423.36,26.7954,4.033,2.6974,0.002633,22207.5,8482.5,45.547583,17.397607,null,0.48909,…,3.399677,3.595702,0.009897,0.084978,0.078944,2017-05-12,236,2017-10-23,-71,2023-04

In [ ]:
b